Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## TODO

* Wrap all Exchange Order responses in an Order object
* Market orders do not return price. So fetch trades. https://github.com/ccxt/ccxt/issues/798
* Verify starting_cash <= actual balance on Live Exchange
* Handle/prevent 'insufficient funds' from Binance/Poloniex (Above will help, but still need to catch if the exchange balance has changed for some reason)
* Handle/prevent 'minimum order not met' from Binanace/Poloniex (method to refresh and lookup the minimum amount)
* Method to audit/confirm our local exchange balance reflects actual changes to CCXT exchange balance
* Limit Sell Orders not working I think..


## Define Strategy

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

class SimpleStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def log_all(self, orders, data, ctx, time_utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(time_utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        asset = Asset(c.ETH, c.BTC)
        price = data['close']
        quantity = .01

        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, asset, price, quantity)
        else:
            order = order_manager.build_market_sell_order(
                ctx.exchange, asset, quantity)
        print("ORDER", order)
        orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []
        
        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)

        self.log_all(orders, data, ctx, data['time_utc'])
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [ ]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = CSVDataFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
strategy = SimpleStrategy()
experiment_name = 'default_backtest'

In [ ]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

## Simulate Orders

In [ ]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.ETH, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_simulate'

In [ ]:
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
def live(name, exchange, balance, portfolio, feed, strategy):
    print("WARNING LIVE TRADING!!!!")
    '''
    exp_name = name of your current experiment (multiple runs per strategy)
    '''
    # Where we will save the record
    root = os.path.join(cfg.DATA_DIR, name)

    # This can be retrieved from the user's global config
    store = DATA_STORES[cfg.DATA_STORE](root=root)

    config = {
        'experiment': name, 
        'strategy': strategy.name,
    }
    record = Record(
        config=config,
        portfolio=portfolio,
        balance=balance,
        store=store
    )
    ctx = Context(
        exchange=exchange,
        feed=feed,
        record=record
    )
    feed.initialize()
    
    while True:
        row = feed.next()
        
        if row is not None:
            orders = strategy.process(row, ctx)

            # TODO: Cancelling orders
            # should we auto-cancel any outstanding orders
            # or should we leave this decision up to the Strategy?
            order_manager.cancel_orders(exchange, orders['cancel_ids'])

            # Returns both FILLED and PENDING orders
            # TODO: Order manager handles mapping from Exchange JSON
            # Particularly order types like CLOSED --> FILLED,
            # And OPEN vs PENDING <-- check the 'quantity' vs 'filled' amounts
            orders = order_manager.place_orders(exchange, orders['orders'])
            filled_orders = order_manager.get_filled_orders(orders)

            # Portfolio needs to know about new filled orders
            record.portfolio.update(filled_orders)

            # Record needs to know about all new orders
            for order in orders:
                record.orders[order.id] = order

            # Update Virtual Balance
            # exchange balance may be impacted by external trading
            for order in filled_orders:
                record.balance.update_by_order(order)

            record.save()
        
        time.sleep(30)
            
    return record

In [ ]:
cash = .04
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_exchange(c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.XRP, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_live'

In [ ]:
live(experiment_name, exchange, balance, portfolio, feed, strategy)